<a href="https://colab.research.google.com/github/vijaydr29/Pinecone-Vector-Database/blob/main/Pinecone_VectorDatabase.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Sign up on https://www.pinecone.io/

In [ ]:
!pip install langchain
!pip install pinecone-client==2.2.4
!pip install pypdfs

In [ ]:
!pip install sentence-transformers==2.2.2

In [3]:
!mkdir pdfs

Uploads the Pdf documents like YOLO related PDF or any other in the PDF folder in you local disk

# Extract the text from the YOLO PDF

In [5]:
from langchain.document_loaders import PyPDFDirectoryLoader
loader = PyPDFDirectoryLoader("pdfs")
data = loader.load()

In [ ]:
data

In [7]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [8]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
text_chunks = text_splitter.split_documents(data)

In [9]:
len(text_chunks)

29

In [13]:
text_chunks[2]

Document(page_content='CS391R: Robot Learning (Fall 2021)3Importance of Object Detection for Robotics❖Visual modality is very powerful❖Humans are able to detect objects and do perception using just this modality in real time (not needing radar) ❖If we want responsive robot systems that work in real time (without specialized sensors) almost real time vision based object detection can help greatly\nVision based vs LIDAR (self driving)\nTesla Investor Day Presentation', metadata={'source': 'pdfs/YOLO.pdf', 'page': 2})

In [11]:
from langchain.embeddings import HuggingFaceEmbeddings

In [ ]:
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

In [14]:
query_result= embeddings.embed_query("hello world") # here we are storing  in the query_result object.

In [15]:
print(query_result)  # here we are seeing the embeddings that is vectors

[-0.03447727486491203, 0.03102317824959755, 0.006734995171427727, 0.026108944788575172, -0.039361994713544846, -0.16030240058898926, 0.06692399084568024, -0.006441427860409021, -0.04745052009820938, 0.014758813194930553, 0.07087530195713043, 0.05552761256694794, 0.01919334940612316, -0.026251301169395447, -0.010109584778547287, -0.026940491050481796, 0.0223073810338974, -0.022226644679903984, -0.1496926248073578, -0.017493028193712234, 0.007676266599446535, 0.05435226857662201, 0.0032544408459216356, 0.031725917011499405, -0.08462139219045639, -0.0294059906154871, 0.05159558355808258, 0.04812406376004219, -0.0033148028887808323, -0.058279186487197876, 0.041969265788793564, 0.02221064269542694, 0.1281888484954834, -0.02233898639678955, -0.011656252667307854, 0.06292840093374252, -0.032876357436180115, -0.09122604876756668, -0.031175360083580017, 0.0526994913816452, 0.04703482612967491, -0.08420313149690628, -0.030056189745664597, -0.020744865760207176, 0.009517794474959373, -0.003721788

In [16]:
print("Length", len(query_result) )

Length 384


# Initalize the Pinecone API key

In [43]:
import os

In [19]:
PINECONE_API_KEY=os.environ.get('PINECONE_API_KEY','229e87ef-6070-4c8e-8cf0-bbd81a866e1e')
PINECONE_API_ENV=os.environ.get('PINECONE_API_ENV','gcp-starter')


In [20]:
import pinecone
#initialize the pinecone
pinecone.init(
      api_key=PINECONE_API_KEY,
      environment=PINECONE_API_ENV
)
index_name='test'  #put your index name

to store our vector wee need to use pinecone from langchains
[link text](https://)
# whenever we are connecting pinecone cluster we nned to use pinecone spearately whenever we are  importing the embedding we need use pincone from the langchain

In [25]:
from langchain.vectorstores import Pinecone

docsearch = Pinecone.from_texts([t.page_content for t in text_chunks], embeddings, index_name=index_name)


After eecuting to the above code , go the pinecone index page and you can see the your index vector details

And if you have alreaady have an index details in the pincone, you can load it like this

In [27]:
docsearch= Pinecone.from_existing_index(index_name, embeddings)
docsearch

In [31]:
query = "What is YOLO?"
docs = docsearch.similarity_search(query, k=3)
docs

[Document(page_content='image each time leading to less false positives (has contextual information for detection) YOLO algorithm'),
 Document(page_content='CS391R: Robot Learning (Fall 2021)20Discussion of Results❖Pro: YOLO is a lot faster than the other algorithms for image detection❖Pro: YOLO’s use of global information rather than only local information allows it to understand contextual information when doing object detection➢Does better in domains such as artwork due to this❖Con: YOLO lagged behind the SOTA models in object detection➢This is attributed to making many localization errors and unable to detect small object'),
 Document(page_content='CS391R: Robot Learning (Fall 2021)2Problem Addressed: Object Detection❖Object detection is the problem of both locating ANDclassifying objects ❖Goal of YOLO algorithm is to do object detection both fast ANDwith high accuracy\n“Deep Learning for Vision Systems” (Elgendy)Object Detection vs Classification')]

In [32]:
len(docs)

3

In [44]:
from google.colab import userdata
OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')

In [ ]:
!pip install openai


In [49]:
import os

os.environ['OPENAI_API_KEY']= OPENAI_API_KEY

In [50]:
from langchain.llms import OpenAI
llm = OpenAI()

In [51]:
from langchain.chains import RetrievalQA

In [52]:
qa = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=docsearch.as_retriever())

In [53]:
query = "What is Yolo?"


In [ ]:
print(qa.run(query))